In [38]:
# Importy
# Konfiguracja środowiska i przygotowanie danych
!pip install sqlalchemy cx_Oracle pandas
#!pip install seaborn
#!pip install matplotlib
#!pip install plotly
#!pip install redis
!pip install pandas
!pip install pymongo
from sqlalchemy import create_engine
import sqlalchemy as sa
# Importowanie bibliotek
from datetime import datetime
import pandas as pd
from pymongo import MongoClient
import pprint
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#from IPython.display import display
import cx_Oracle
import json

# Konfiguracja bazy danych
username = 'S101095'
password = 'S101095'
host = '217.173.198.135'
port = '1521'
service_name = 'tpdb'

dsn = f'{username}:{password}@{host}:{port}/?service_name={service_name}'
 
engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={service_name}')
connection = engine.connect()

# Połączenie z bazą danych

# Połączenie z bazą MongoDB
client = MongoClient("mongodb://localhost:50001/")
print(client.list_database_names()) 
db = client["mongo-server"] 
collection = db["user_data"]  


# Wstawianie przykładowych danych do MongoDB
data = [
    {
        "username": "jankowalski@wp.pl",
        "weight": [
            {"date": datetime(2024, 11, 1), "value": 70.5},
            {"date": datetime(2024, 12, 1), "value": 71.0}
        ],
        "bmi": [
            {"date": datetime(2024, 11, 1), "value": 22.3},
            {"date": datetime(2024, 12, 1), "value": 22.5}
        ],
        "price": [
            {"date": datetime(2024, 11, 1), "value": 3.5},
            {"date": datetime(2024, 12, 1), "value": 3.6}
        ]
    },
    {
        "username": "annanowak@wp.pl",
        "weight": [
            {"date": datetime(2024, 11, 1), "value": 65.0},
            {"date": datetime(2024, 12, 1), "value": 64.8}
        ],
        "bmi": [
            {"date": datetime(2024, 11, 1), "value": 21.5},
            {"date": datetime(2024, 12, 1), "value": 21.4}
        ],
        "price": [
            {"date": datetime(2024, 11, 1), "value": 3.8},
            {"date": datetime(2024, 12, 1), "value": 3.7}
        ]
    },
        {
        "username": "dawidek@wp.pl",
        "weight": [
            {"date": datetime(2024, 11, 1), "value": 85.0},
            {"date": datetime(2024, 12, 1), "value": 84.8}
        ],
        "bmi": [
            {"date": datetime(2024, 11, 1), "value": 25.5},
            {"date": datetime(2024, 12, 1), "value": 24.4}
        ],
        "price": [
            {"date": datetime(2024, 11, 1), "value": 38},
            {"date": datetime(2024, 12, 1), "value": 37}
        ]
    },
        {
        "username": "anonim@wp.pl",
        "weight": [
            {"date": datetime(2024, 11, 1), "value": 111.0},
            {"date": datetime(2024, 12, 1), "value": 109.8}
        ],
        "bmi": [
            {"date": datetime(2024, 11, 1), "value": 35.5},
            {"date": datetime(2024, 12, 1), "value": 34.4}
        ],
        "price": [
            {"date": datetime(2024, 11, 1), "value": 25},
            {"date": datetime(2024, 12, 1), "value": 15}
        ]
    }
]

# Wstawianie danych do kolekcji
collection.insert_many(data)

print('')
print('A. Wyszukiwanie pojedynczych dokumentów')
print('1. Wyszukiwanie danych użytkownika "jankowalski@wp.pl"')
user = collection.find_one({"username": "jankowalski@wp.pl"})
print(user)
print('')

print('2. Wyszukiwanie danych o wadze z konkretnego dnia')
weight_entry = collection.find_one({"username": "jankowalski@wp.pl"}, {"weight": {"$elemMatch": {"date": datetime(2024, 11, 1)}}})
print(weight_entry)
print('')

print('3. Wyszukiwanie danych BMI dla "annanowak@wp.pl"')
bmi_entry = collection.find_one({"username": "annanowak@wp.pl"}, {"bmi": 1})
print(bmi_entry)
print('')

print('B. Wyszukiwanie grup dokumentów')
print('1. Wyszukiwanie wszystkich użytkowników')
users = collection.find({}, {"username": 1})
for user in users:
    print(user)
print('')

print('2. Wyszukiwanie użytkowników o BMI powyżej 22.0')
users_with_high_bmi = collection.find({"bmi.value": {"$gt": 22.0}}, {"username": 1})
for user in users_with_high_bmi:
    print(user)
print('')

print('3. Wyszukiwanie użytkowników z wagą poniżej 70')
users_with_low_weight = collection.find({"weight.value": {"$lt": 70}}, {"username": 1})
for user in users_with_low_weight:
    print(user)
print('')

user = collection.find_one({"username": "jankowalski@wp.pl"})

print('C. Wyliczenia wartości')
print('1. Obliczanie średniego BMI użytkownika "jankowalski@wp.pl"')
bmi_values = [entry["value"] for entry in user['bmi']]
avg_bmi = sum(bmi_values) / len(bmi_values)
print(f"Średnie BMI użytkownika 'jankowalski@wp.pl': {avg_bmi}")
print('')

print('2. Obliczanie maksymalnej wagi wśród wszystkich użytkowników')
max_weight = collection.aggregate([
    {"$unwind": "$weight"},
    {"$group": {"_id": None, "max_weight": {"$max": "$weight.value"}}}
])
print(list(max_weight))
print('')

print('3. Liczba wpisów dla każdego użytkownika')
entry_counts = collection.aggregate([
    {"$project": {"username": 1, "total_entries": {"$size": {"$concatArrays": ["$weight", "$bmi", "$price"]}}}}
])
for count in entry_counts:
    print(count)
print('')

print('D. Agregacje wyników')
print('1. Suma wszystkich wpisów dla wszystkich użytkowników')
total_entries = collection.aggregate([
    {"$unwind": {"path": "$weight", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$bmi", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$price", "preserveNullAndEmptyArrays": True}},
    {"$group": {"_id": None, "total_weight": {"$sum": 1}, "total_bmi": {"$sum": 1}, "total_price": {"$sum": 1}}}
])
print(list(total_entries))
print('')

print('2. Liczba użytkowników w bazie')
user_count = collection.distinct("username")
print(f"Liczba użytkowników: {len(user_count)}")
print('')

print('3. Średnia cena z bazy')
avg_price = collection.aggregate([
    {"$unwind": "$price"},
    {"$group": {"_id": None, "avg_price": {"$avg": "$price.value"}}}
])
print(list(avg_price))


['admin', 'config', 'local']

A. Wyszukiwanie pojedynczych dokumentów
1. Wyszukiwanie danych użytkownika "jankowalski@wp.pl"
{'_id': ObjectId('676057e5772d521b41c85ca9'), 'username': 'jankowalski@wp.pl', 'weight': [{'date': datetime.datetime(2024, 11, 1, 0, 0), 'value': 70.5}, {'date': datetime.datetime(2024, 12, 1, 0, 0), 'value': 71.0}], 'bmi': [{'date': datetime.datetime(2024, 11, 1, 0, 0), 'value': 22.3}, {'date': datetime.datetime(2024, 12, 1, 0, 0), 'value': 22.5}], 'price': [{'date': datetime.datetime(2024, 11, 1, 0, 0), 'value': 3.5}, {'date': datetime.datetime(2024, 12, 1, 0, 0), 'value': 3.6}]}

2. Wyszukiwanie danych o wadze z konkretnego dnia
{'_id': ObjectId('676057e5772d521b41c85ca9'), 'weight': [{'date': datetime.datetime(2024, 11, 1, 0, 0), 'value': 70.5}]}

3. Wyszukiwanie danych BMI dla "annanowak@wp.pl"
{'_id': ObjectId('676057e5772d521b41c85caa'), 'bmi': [{'date': datetime.datetime(2024, 11, 1, 0, 0), 'value': 21.5}, {'date': datetime.datetime(2024, 12, 1, 0, 0), 'v